# Reactome_GNN Demo

This notebook shows how to obtain GNN-generated embeddings for nodes in the human pathway network.

In [1]:
import pickle
import torch
import dgl
from reactome_gnn import utils, model

Using backend: pytorch


## Embeddings of a single graph

Specify the list of markers and the threshold p-value.

In [2]:
markers = ['RAS', 'MAP', 'IL10', 'EGF', 'EGFR', 'STAT']
p_value = 0.05
study = 'demo'

Create network from the markers and with specified p-value threshold, given the name of the study. This function first runs the enrichment analysis on the human pathway network and returns the significant nodes. This information is then used to create the graph.

In [3]:
graph = utils.create_network_from_markers(markers, p_value, study)

The main class attribute of the returned graph is `graph.graph_nx` which is a `NetworkX.DiGraph` object---a directed graph where each node has a weight and a significance. Weight is the p-value returned by the enrichment analysis, and significance is a string stating whether the pathway in the network is significant or not. Significant pathways are those that have p-value lower than the specified threshold, in this case 0.05.

For example, let's take a look at the nodes corresponding to R-HSA-1358803 and R-HSA-15869 pathways:

In [4]:
graph.graph_nx.nodes['R-HSA-1358803']

{'stId': 'R-HSA-1358803',
 'name': 'Downregulation of ERBB2:ERBB3 signaling',
 'weight': 0.0109,
 'significance': 'significant'}

In [5]:
graph.graph_nx.nodes['R-HSA-15869']

{'stId': 'R-HSA-15869',
 'name': 'Metabolism of nucleotides',
 'weight': 0.161,
 'significance': 'non-significant'}

We notice that R-HSA-1358803 corresponds to "Downregulation of ERBB2:ERBB3 signaling", and is significant in the network created with the markers specified above (its p-value is 0.0109 which is less than 0.05). On the other hand, R-HSA-15869 corresponds to "Metabolism of nucleotides" which is non-significant in this network (its p-value is 0.161).

Next we want to do is create an embedding of that graph. But in order to do that, graph should be in the DGL format.

In [6]:
graph_dgl = utils.nx_to_dgl(graph.graph_nx)
graph_dgl

Graph(num_nodes=16050, num_edges=16850,
      ndata_schemes={'weight': Scheme(shape=(), dtype=torch.float32), 'significance': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={})

We also need a GNN model which will produce the embeddings. Since we want the model to be reusable at any given time, the parameters of the model have to be saved to the disk and loaded upon model initialization. The demo model that is already saved is a two-layers Graph Convolutional Network (GCN) and creates the the embeddings with latent dimension of 8.

The GCN is chosen because it is a relatively "lightweight" model that was proven to have good performance on various graph-related tasks. For example, even when comparing an untrained GCN network against DeepWalk on semi-supervised node classification task, untrained GCN had superior performance! This means that it can more successfully leverage the topology of a graph and create meaningful representation than the traditional methods.

In [7]:
net = model.GCNModel(dim_latent=8, num_layers=2)
net.load_state_dict(torch.load('data/example/models/model.pth'))

print(net)

GCNModel(
  (embedder): Embedding(2, 2)
  (linear): Linear(in_features=3, out_features=8, bias=True)
  (conv_0): GraphConv(in=8, out=8, normalization=both, activation=None)
  (relu): LeakyReLU(negative_slope=0.01)
  (layers): ModuleList(
    (0): GraphConv(in=8, out=8, normalization=both, activation=None)
  )
)


In [8]:
embeddings = net(graph_dgl).detach()
print(embeddings.shape)
embeddings

torch.Size([16050, 8])


tensor([[ 0.0569,  0.3340, -0.0890,  ..., -0.2442, -0.3875, -0.4222],
        [ 0.0569,  0.3340, -0.0890,  ..., -0.2442, -0.3875, -0.4222],
        [ 0.0577,  0.3345, -0.0893,  ..., -0.2480, -0.3947, -0.4272],
        ...,
        [ 0.0763,  0.4480, -0.1193,  ..., -0.3276, -0.5198, -0.5664],
        [ 0.0808,  0.4744, -0.1264,  ..., -0.3469, -0.5503, -0.5997],
        [ 0.0777,  0.4559, -0.1215,  ..., -0.3334, -0.5289, -0.5764]])

This tensor has a shape 16050 x 8, which means that each row represents the embedding of one node, as there are 16050 nodes in the pathway network, and each of these embeddings is 8-dimensional (as specified when initializing the model).

To find the embeddings of some specific pathways, e.g. R-HSA-1358803, we have to load some auxilary information. We can also fetch embeddings of pathways by specifying their name, not only stId.

In [9]:
name_to_id = pickle.load(open('data/example/info/name_to_id.pkl', 'rb'))
sorted_stid_list = pickle.load(open('data/example/info/sorted_stid_list.pkl', 'rb'))

Here, `name_to_id` is a dictionary which maps each pathway name to its stId, and sorted_stid_list is, as the name states, the sorted list of all the stIds in the network. This is necessary due to how DGL stores the nodes when converting the network from the NetworkX format. Let's first obtain the embedding of R-HSA-1358803:

In [10]:
idx = sorted_stid_list.index('R-HSA-1358803')
embeddings[idx]

tensor([-0.0194, -0.0138, -0.0101, -0.0111, -0.1170, -0.0788, -0.1274, -0.0852])

Let's now get the embedding of the pathway "Metabolism of nucleotides":

In [11]:
stid = name_to_id['Metabolism of nucleotides']
idx = sorted_stid_list.index(stid)
embeddings[idx]

tensor([ 0.0160,  0.0677, -0.0214, -0.0563, -0.1249, -0.0655, -0.1089, -0.1076])

## Working with datasets

Steps above are useful when we are working with a single pathway network, but the procedure could be cumbersome when we need to work with several networks. That's where we can use datasets to automate part of the process. Here we will work with a toy dataset consisting of four different pathways, generated by lighting up pathways specified via their name. The networks are called A, B, C, and D, and the relations between them is such that $A = D$, $A \& D \subset C$, $B \cap C = \emptyset$.

In [12]:
graph_A, graph_B, graph_c, graph_D = utils.create_toy_study_with_names(data_dir='data/example')
embedding_dict = utils.create_embeddings(data_dir='data/example')

What happens under the hood is that a PathwayDataset is created. This is a DGL dataset that makes processing raw graphs and loading DGL graphs easier. Creating this dataset is not mandatory, but helps when working with multiple graphs.

In [13]:
from reactome_gnn import dataset

ds = dataset.PathwayDataset(root='data/example')

graph, name = ds[0]
print(name)
print(graph)

study_A.dgl
Graph(num_nodes=16050, num_edges=16850,
      ndata_schemes={'significance': Scheme(shape=(), dtype=torch.float32), 'weight': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={})


But let's return to creating the embeddings. What's inside the embedding_dict?

In [14]:
embedding_dict

{'study_A.dgl': tensor([[ 0.0569,  0.3340, -0.0890,  ..., -0.2442, -0.3875, -0.4222],
         [ 0.0569,  0.3340, -0.0890,  ..., -0.2442, -0.3875, -0.4222],
         [ 0.0583,  0.3424, -0.0912,  ..., -0.2504, -0.3973, -0.4329],
         ...,
         [ 0.0763,  0.4480, -0.1193,  ..., -0.3276, -0.5198, -0.5664],
         [ 0.0808,  0.4744, -0.1264,  ..., -0.3469, -0.5503, -0.5997],
         [ 0.0777,  0.4559, -0.1215,  ..., -0.3334, -0.5289, -0.5764]]),
 'study_B.dgl': tensor([[ 0.0569,  0.3340, -0.0890,  ..., -0.2442, -0.3875, -0.4222],
         [ 0.0569,  0.3340, -0.0890,  ..., -0.2442, -0.3875, -0.4222],
         [ 0.0583,  0.3424, -0.0912,  ..., -0.2504, -0.3973, -0.4329],
         ...,
         [ 0.0763,  0.4480, -0.1193,  ..., -0.3276, -0.5198, -0.5664],
         [ 0.0808,  0.4744, -0.1264,  ..., -0.3469, -0.5503, -0.5997],
         [ 0.0777,  0.4559, -0.1215,  ..., -0.3334, -0.5289, -0.5764]]),
 'study_C.dgl': tensor([[ 0.0569,  0.3340, -0.0890,  ..., -0.2442, -0.3875, -0.4222],


We see that we have obtained a dictionary, where keys are the names of the graphs, and the values are the embedding tensors for the corresponding graphs. We can obtain the embedding of each node in the same way as we did above, in the case of a single graph.

### Comparing the embeddings

To compare the embeddings obtained above, we use Canonical Correlation Analysis (CCA).  First we specify the targets Y for each graph, which specify which nodes are significant and which are not. Then we fit the data on the CCA model from scikit-learn. This fitted model can be used to transform embedding data and compare embeddings of nodes across different graphs. This is all done inside the `utils.fit_cca_on_toy_data()` function, so we can simply call it like this:

In [15]:
cca = utils.fit_cca_on_toy_data('data/example')

But for the sake of ilustration, we show here how it is done step-by-step so that the code can be reused with new data.

In [16]:
# Note: We do not fit graph D, since it is equivalent to graph A

from sklearn.cross_decomposition import CCA

study_A = ["Signaling by WNT", "WNT ligand biogenesis and trafficking", "Degradation of beta-catenin by the destruction complex",
           "TCF dependent signaling in response to WNT", "Beta-catenin independent WNT signaling"]
study_B = ["Autophagy", "Macroautophagy", "Chaperone Mediated Autophagy", "Late endosomal microautophagy"]
study_C = ["Signal Transduction", "Signaling by NOTCH", "Signaling by NOTCH1", "Signaling by NOTCH2", "Signaling by NOTCH3",
           "Signaling by NOTCH4", "Activated NOTCH1 Transmits Signal to the Nucleus",
           "NOTCH1 Intracellular Domain Regulates Transcription", "Signaling by WNT", "WNT ligand biogenesis and trafficking",
           "Degradation of beta-catenin by the destruction complex", "TCF dependent signaling in response to WNT",
           "Beta-catenin independent WNT signaling"]

emb_A = embedding_dict['study_A.dgl']
emb_B = embedding_dict['study_B.dgl']
emb_C = embedding_dict['study_C.dgl']
emb_D = embedding_dict['study_D.dgl']

name_to_id = pickle.load(open('data/example/info/name_to_id.pkl', 'rb'))
sorted_stid_list = pickle.load(open('data/example/info/sorted_stid_list.pkl', 'rb'))

indices_A = [sorted_stid_list.index(id) for name, id in name_to_id.items() if name in study_A]
indices_B = [sorted_stid_list.index(id) for name, id in name_to_id.items() if name in study_B]
indices_C = [sorted_stid_list.index(id) for name, id in name_to_id.items() if name in study_C]

y_A = torch.tensor([1.0 if i in indices_A else 0.0 for i in range(len(sorted_stid_list))]).unsqueeze(-1)
y_B = torch.tensor([1.0 if i in indices_B else 0.0 for i in range(len(sorted_stid_list))]).unsqueeze(-1)
y_C = torch.tensor([1.0 if i in indices_C else 0.0 for i in range(len(sorted_stid_list))]).unsqueeze(-1)

cca = CCA(1)
cca.fit(emb_A, y_A).fit(emb_B, y_B).fit(emb_C, y_C)
cca

CCA(n_components=1)

Next thing we need to do is transform the embedding tensors:

In [17]:
emb_A_t = cca.transform(emb_A)
emb_B_t = cca.transform(emb_B)
emb_C_t = cca.transform(emb_C)
emb_D_t = cca.transform(emb_D)

Finally, we can compare the obtained embeddings. For example, let's compare the embeddings of nodes that correspond to pathway "WNT ligand biogenesis and trafficking". This pathway is present in graphs A, C, and D, but not in B.

In [18]:
stid = name_to_id['WNT ligand biogenesis and trafficking']
idx = sorted_stid_list.index(stid)

print(emb_A_t[idx], emb_B_t[idx], emb_C_t[idx], emb_D_t[idx])

[0.00019627] [-6.02214974e-07] [0.00028198] [0.00019627]


We can see that the values returned are identical in the case of A and D, as it should be since these networks are the same. We also notice that the values are relatively close to each other in case of A and C (as well as D and C), because this pathway is significant in all three networks. Finally, we notice that the value is significantly different (negative, even) in the case of network B in which this pathway is not significant. This shows that the embeddings are consistent within this dataset.